#Importações

**BIBLIOTECAS**

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

In [ ]:
import pandas as pd

**CARREGANDO DATAFRAME PAISES** 
DATASETS EXTRAÍDOS DO KAGGLE NO DIA 17/11/2021

In [ ]:
dados_br_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/BR_youtube_trending_data.csv')
dados_ca_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/CA_youtube_trending_data.csv')
dados_de_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/DE_youtube_trending_data.csv')
dados_fr_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/FR_youtube_trending_data.csv')
dados_gb_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/GB_youtube_trending_data.csv')
dados_in_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/IN_youtube_trending_data.csv')
dados_jp_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/JP_youtube_trending_data.csv')
dados_kr_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/KR_youtube_trending_data.csv')
dados_mx_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/MX_youtube_trending_data.csv')
dados_us_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/US_youtube_trending_data.csv')
dados_ru_csv = pd.read_csv('gs://projetofinalgrupo8/entrada/RU_youtube_trending_data.csv')

In [ ]:
dados_br_csv.shape

(93793, 16)

#Criação de colunas, exploração e união dos Datasets

**ADICIONANDO COLUNA COM ABREVIAÇÃO DO PAÍS**

In [ ]:
dados_br_csv['country'] = 'BR'
dados_ca_csv['country'] = 'CA'
dados_de_csv['country'] = 'DE'
dados_fr_csv['country'] = 'FR'
dados_gb_csv['country'] = 'GB'
dados_in_csv['country'] = 'IN'
dados_jp_csv['country'] = 'JP'
dados_kr_csv['country'] = 'KR'
dados_mx_csv['country'] = 'MX'
dados_us_csv['country'] = 'US'
dados_ru_csv['country'] = 'RU'

**VERIFICANDO DIMENSÃO DE CADA PAÍS**

In [ ]:
print(f' BR: {dados_br_csv.shape},\n CA: {dados_ca_csv.shape},\n DE: {dados_de_csv.shape},\n FR: {dados_fr_csv.shape},\n GB: {dados_gb_csv.shape},\n IN: {dados_in_csv.shape},\n JP: {dados_jp_csv.shape},\n KR: {dados_kr_csv.shape},\n MX: {dados_mx_csv.shape},\n US: {dados_us_csv.shape},\n RU: {dados_ru_csv.shape}')

 BR: (93793, 17),
 CA: (93744, 17),
 DE: (93743, 17),
 FR: (93791, 17),
 GB: (93795, 17),
 IN: (88182, 17),
 JP: (93789, 17),
 KR: (90754, 17),
 MX: (93599, 17),
 US: (93791, 17),
 RU: (92641, 17)


**CONCATENANDO DADOS WORLD**

In [ ]:
dataframes = [dados_br_csv, dados_ca_csv, dados_de_csv, dados_fr_csv, dados_gb_csv, dados_in_csv, dados_jp_csv, 
              dados_kr_csv, dados_mx_csv, dados_us_csv, dados_ru_csv]
dados_world_csv = pd.concat(dataframes)

**VERIFICANDO DADOS WORLD**

In [ ]:
dados_world_csv.shape

(1021622, 17)

**CARREGANDO JSON COMPLEMENTAR DE PARA DADOS WORLD**

In [ ]:
dados_json = pd.read_json('gs://projetofinalgrupo8/entrada/BR_category_id.json')

**EXPLORANDO JSON**

In [ ]:
dados_json.sample()

,kind,etag,items
14,youtube#videoCategoryListResponse,kBCr3I9kLHHU79W4Ip5196LDptI,"{'kind': 'youtube#videoCategory', 'etag': 'hHU..."


In [ ]:
dados_json['items'][0]

{'etag': 'IfWa37JGcqZs-jZeAyFGkbeh6bc',
 'id': '1',
 'kind': 'youtube#videoCategory',
 'snippet': {'assignable': True,
  'channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'title': 'Film & Animation'}}

**DESCOMPRIMINDO JSON**

In [ ]:
new_dados = []

for i in range(len(dados_json['items'])):
  #print(dados_json['items'][i])
  new_data = {}
  new_data['json_kind'] = dados_json['items'][i]['kind']
  new_data['json_etag'] = dados_json['items'][i]['etag']
  new_data['categoryId'] = dados_json['items'][i]['id']
  new_data['json_title'] = dados_json['items'][i]['snippet']['title']
  new_data['json_assignable'] = dados_json['items'][i]['snippet']['assignable']
  new_data['json_channelId'] = dados_json['items'][i]['snippet']['channelId']
  new_dados.append(new_data)

In [ ]:
new_df = pd.DataFrame(new_dados)

In [ ]:
new_df.dtypes

json_kind          object
json_etag          object
categoryId         object
json_title         object
json_assignable      bool
json_channelId     object
dtype: object

In [ ]:
new_df['categoryId'] = new_df['categoryId'].astype('int64')

In [ ]:
new_df.dtypes

json_kind          object
json_etag          object
categoryId          int64
json_title         object
json_assignable      bool
json_channelId     object
dtype: object

**COMBINANDO DADOS WORLD COM JSON**

In [ ]:
data_raw = pd.merge(dados_world_csv, new_df, on=['categoryId'], how='left')

In [ ]:
data_raw.shape

(1021622, 22)

In [ ]:
data_raw.head(3)

,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description,country,json_kind,json_etag,json_title,json_assignable,json_channelId
0,s9FH4rDMvds,LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!,2020-08-11T22:21:49Z,UCGfBwrCoi9ZJjKiUK8MmJNw,Pietro Guedes,22,2020-08-12T00:00:00Z,pietro|guedes|ingrid|ohara|pingrid|vlog|amigos...,263835,85095,487,4500,https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg,False,False,"Salve rapaziada, neste vídeo me declarei pra e...",BR,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ
1,jbGRowa5tIk,ITZY “Not Shy” M/V TEASER,2020-08-11T15:00:13Z,UCaO6TYtlC8U5ttz62hTrZgg,JYP Entertainment,10,2020-08-12T00:00:00Z,JYP Entertainment|JYP|ITZY|있지|ITZY Video|ITZY ...,6000070,714310,15176,31040,https://i.ytimg.com/vi/jbGRowa5tIk/default.jpg,False,False,ITZY Not Shy M/V[ITZY Official] https://www.yo...,BR,youtube#videoCategory,HCjFMARbBeWjpm6PDfReCOMOZGA,Music,True,UCBR8-60-B28hp2BmDPdntcQ
2,3EfkCrXKZNs,Oh Juliana PARÓDIA - MC Niack,2020-08-10T14:59:00Z,UCoXZmVma073v5G1cW82UKkA,As Irmãs Mota,22,2020-08-12T00:00:00Z,OH JULIANA PARÓDIA|MC Niack PARÓDIA|PARÓDIAS|A...,2296748,39761,5484,0,https://i.ytimg.com/vi/3EfkCrXKZNs/default.jpg,True,False,Se inscrevam meus amores! 📬 Quer nos mandar al...,BR,youtube#videoCategory,QMEBz6mxVdklVaq8JwesPEw_4nI,People & Blogs,True,UCBR8-60-B28hp2BmDPdntcQ


In [ ]:
data_raw.dtypes

video_id             object
title                object
publishedAt          object
channelId            object
channelTitle         object
categoryId            int64
trending_date        object
tags                 object
view_count            int64
likes                 int64
dislikes              int64
comment_count         int64
thumbnail_link       object
comments_disabled      bool
ratings_disabled       bool
description          object
country              object
json_kind            object
json_etag            object
json_title           object
json_assignable      object
json_channelId       object
dtype: object

#Removendo colunas
Optamos por remover essas colunas por serem relevantes para o nosso projeto

In [ ]:
data_raw2 = data_raw.drop(['categoryId', 'tags', 'thumbnail_link', 'description', 'json_kind', 'json_etag', 'json_assignable', 'json_channelId'], axis=1)

In [ ]:
data_raw2.shape

(1021622, 14)

#Exportando para o GCP

In [ ]:
from google.cloud import storage
import os
serviceAccount = '/content/projetofinalgrupo8-2dcd866c3f46.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount


client = storage.Client()
bucket = client.get_bucket('projetofinalgrupo8')
    
bucket.blob('saida/youtube_data_base.csv').upload_from_string(data_raw2.to_csv(index=False), 'text/csv')